# Data Preprocessing

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
import numpy as np
import networkx as nx
import sentencepiece as spm

In [ ]:
data_path = os.path.join("Liu_Kheyer_Retrosynthesis_Data")
train_path = os.path.join(data_path, "train")
validation_path = os.path.join(data_path, "validation")
test_path = os.path.join(data_path, "test")

 readme.txt   train	  train_40x   validation        vocab2.txt   vocab.gdoc
 test	      train_16x   train_4x   'vocab (1).gdoc'   vocab3.txt   vocab.txt


In [ ]:
import pandas as pd
train_reactants_df = pd.read_csv(os.path.join(
    train_path, "train_targets.txt"), header=None)
train_products_df = pd.read_csv(os.path.join(
    train_path, "train_sources.txt"), header=None)

In [ ]:
def process_dfs(react_df, prod_df):
    react_df_n = react_df.copy()
    prod_df_n = prod_df.copy()
    prod_df_n.rename(columns={0: "products"}, inplace=True)
    react_df_n.rename(columns={0: "reactants"}, inplace=True)
    prod_df_n["reaction_type"] = prod_df_n["products"].str.extract(
        r"(\<RX_.*\>)")
    prod_df_n["products"] = prod_df_n["products"].str.replace(
        r"(\<RX_.*\>)", "", regex=True)
    df = pd.concat([react_df_n, prod_df_n], axis=1)
    return df


df = process_dfs(train_reactants_df, train_products_df)
rxns = df["reaction_type"].value_counts(ascending=False)/df.shape[0]
rxns.plot.bar()
rxns

In [ ]:
from rdkit.Chem import rdChemReactions
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
rxn_10 = df[df.reaction_type == "<RX_10>"].full_input_format.tolist()
mol1 = rdChemReactions.ReactionFromSmarts(rxn_10[60])
print(rxn_10[60])
mol1

In [ ]:
def prepare_whole_reaction_padded(df):
    df["full_input_format_delimited"] = "[BOS]" + \
        df["reactants"]+">>"+df["products"]+"[EOS]"
    df["full_input_format_delimited"] = df["full_input_format_delimited"].str.replace(
        " ", "")
    df["full_input_format"] = df["reactants"].str.replace(
        " ", "")+">>"+df["products"].str.replace(" ", "")


prepare_whole_reaction_padded(df)
plt.hist(df["full_input_format_delimited"].str.len())
plt.show()
df.loc[0, "full_input_format_delimited"]

In [ ]:
(df["full_input_format_delimited"].str.len() > 200).sum()

640

We need first to create the vocabulary for the training and target data:

In [ ]:
spt_1 = spm.SentencePieceTrainer.train(
    "--input=Liu_Kheyer_Retrosynthesis_Data/vocab2.txt --model_prefix=m  --user_defined_symbols=[BOS],[EOS],[PAD],. --vocab_size=56 --bos_id=-1 --eos_id=-1")
sp = spm.SentencePieceProcessor()
sp.load('m.model')
print(sp.get_piece_size())
vocabs = [sp.id_to_piece(id) for id in range(sp.get_piece_size())]
print(vocabs)

Let's check the vocabulary created by the trainer by encoding the first example:

In [ ]:
print(len(df["full_input_format_delimited"][0]))
print(sp.encode_as_pieces(df["full_input_format_delimited"][0]))
print(sp.encode_as_ids(df["full_input_format_delimited"][0]))

160
['▁', '[BOS]', 'C', 'S', '(', '=', 'O', ')', '(', '=', 'O', ')', 'O', 'C', '[', 'C', '@', 'H', ']', '1', 'C', 'C', 'C', '(', '=', 'O', ')', 'O', '1', '.', 'F', 'c', '1', 'c', 'c', 'c', '(', 'N', 'c', '2', 'n', 'c', 'n', 'c', '3', 'c', 'c', '(', 'O', 'C', 'C', 'N', '4', 'C', 'C', 'N', 'C', 'C', '4', ')', 'c', '(', 'O', 'C', '4', 'C', 'C', 'C', 'C', '4', ')', 'c', 'c', '2', '3', ')', 'c', 'c', '1', 'C', 'l', '>', '>', 'O', '=', 'C', '1', 'C', 'C', '[', 'C', '@', 'H', ']', '(', 'C', 'N', '2', 'C', 'C', 'N', '(', 'C', 'C', 'O', 'c', '3', 'c', 'c', '4', 'n', 'c', 'n', 'c', '(', 'N', 'c', '5', 'c', 'c', 'c', '(', 'F', ')', 'c', '(', 'C', 'l', ')', 'c', '5', ')', 'c', '4', 'c', 'c', '3', 'O', 'C', '3', 'C', 'C', 'C', 'C', '3', ')', 'C', 'C', '2', ')', 'O', '1', '[EOS]']
[5, 1, 25, 36, 7, 21, 33, 8, 7, 21, 33, 8, 33, 25, 38, 25, 23, 27, 40, 12, 25, 25, 25, 7, 21, 33, 8, 33, 12, 4, 26, 43, 12, 43, 43, 43, 7, 32, 43, 13, 49, 43, 49, 43, 14, 43, 43, 7, 33, 25, 25, 32, 15, 25, 25, 32, 25, 25, 

Convert reaction smarts to fingerprints

In [ ]:
from rdkit import Chem
params = Chem.rdChemReactions.ReactionFingerprintParams()
params.fpSize = 2048
df["rxn"] = df.apply(lambda x: Chem.rdChemReactions.ReactionFromSmarts(
    x["full_input_format"], useSmiles=True), axis=1)
df["fingerprint"] = df.apply(
    lambda x: Chem.rdChemReactions.CreateDifferenceFingerprintForReaction(x["rxn"], params), axis=1)

rdkit.Chem.rdChemReactions.FingerprintType.AtomPairFP

Now we can tokenize the training data:

In [ ]:
PAD_TOKEN = sp.encode_as_ids("[PAD]")[1]
BOS_TOKEN = sp.encode_as_ids("[BOS]")[1]


def tokenize_and_pad(df, max_len=0):
    # choose to set the max_len based on a user entered value or the one derived from the data
    if max_len == 0:
        max_val = df["input_ids"].str.len().sort_values(
            ascending=False).head(1).values[0]
        print(max_val)
        return
    else:
        max_val = max_len
    # Create the encoded columns
    df["input_ids"] = df["full_input_format_delimited"].apply(
        sp.encode_as_ids).apply(lambda x: x[1:])
    df_new = df[df["input_ids"].apply(len) <= max_val].copy()
    print("removed ", df.shape[0]-df_new.shape[0], " samples")

    # calculate how much to pad for each case
    df_new["input_counts"] = max_val-df_new["input_ids"].str.len()

    # pad with the PAD_TOKEN
    df_new["input_ids"] = df_new.apply(lambda row: np.pad(
        row["input_ids"], (0, row["input_counts"]), mode='constant', constant_values=(0, PAD_TOKEN)), axis=1)

    return np.asarray(df_new["input_ids"].values.tolist()), max_val, df_new


train_inputs, train_input_len, df_with_max_200 = tokenize_and_pad(df, 200)
df_with_max_200["input_ids"].str.len().sort_values()

In [ ]:
sp.decode_ids(train_inputs[1].tolist())

'[BOS]COC(=O)c1cc(CCCc2cc3c(=O)[nH]c(N)nc3[nH]2)cs1>>Nc1nc2[nH]c(CCCc3csc(C(=O)O)c3)cc2c(=O)[nH]1[EOS][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

In [ ]:
fingerprints = []
for i in df_with_max_200.index:
    array = np.zeros((0, ), dtype=np.float64)
    Chem.DataStructs.ConvertToNumpyArray(
        df_with_max_200.loc[i, "fingerprint"], array)
    fingerprints.append(array)
len(fingerprints)

In [ ]:
"""df_rxn_10=df[(df["reaction_type"]=="<RX_10>")].copy()
train_inputs,train_input_len,df_rxn_10_max_200=tokenize_and_pad(df_rxn_10,200)
fingerprints=[]
for i in df_rxn_10_max_200.index:
  array=np.zeros((0, ), dtype=np.float64)
  Chem.DataStructs.ConvertToNumpyArray(df_rxn_10_max_200.loc[i,"fingerprint"], array)
  fingerprints.append(array)
with open(os.path.join(train_path,"train_targets_ids_200_rxn10.data"), "w") as f:
  np.savetxt(f, train_inputs,newline='\n')
with open(os.path.join(train_path,"train_targets_ids_fingerprints_200_rxn10.data"), "w") as f:
  np.savetxt(f, fingerprints,newline='\n')

import numpy as np
from sklearn_extra.cluster import KMedoids
from time import time
fingerprints_10=[]
for i in df_with_max_200.index:
  if(df_with_max_200.loc[i,"reaction_type"]=="<RX_10>"):
    array=np.zeros((0, ), dtype=np.float64)
    Chem.DataStructs.ConvertToNumpyArray(df_with_max_200.loc[i,"fingerprint"], array)
    fingerprints_10.append(array)
def find_centroids(,fps):
  centroids=[]
  cluster_els=fps
  nb_of_clusters=int(np.floor(len(cluster_els)*0.05))
  print(len(cluster_els),nb_of_clusters)
  kmed=KMedoids(metric='jaccard',n_clusters=nb_of_clusters, random_state=48,init='k-medoids++').fit(cluster_els)
  centroids.append(kmed.cluster_centers_)
  print("inertia:",kmed.inertia_)
  return np.vstack(centroids)


centroids=find_centroids(np.array(fingerprints_10))
centroids_strings=[]
for i in centroids:
  select=[np.array_equal(i,j) for j in fingerprints]
  if(np.sum(select))>0:
    centroids_strings.append(train_inputs[select][0])
    print(len(centroids_strings))
print(len(centroids_strings))
with open(os.path.join(train_path,"centroids_strings_200_rxn10.data"), "w") as f:
  np.savetxt(f, centroids_strings,newline='\n')
with open(os.path.join(train_path,"centroids_200_rxn10.data"), "w") as f:
  np.savetxt(f, centroids,newline='\n')
"""

181 9
inertia: 84.64804907413179


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


9

In [ ]:
"""import numpy as np
from sklearn_extra.cluster import KMedoids
from time import time
def find_centroids(fps):
    centroids=[]
    cluster_els=fps
    nb_of_clusters=int(np.floor(len(cluster_els)*0.05))
    print(len(cluster_els),nb_of_clusters)
    kmed=KMedoids(metric='jaccard',n_clusters=nb_of_clusters, random_state=48,init='k-medoids++').fit(cluster_els)
    centroids.append(kmed.cluster_centers_)
    print("inertia:",kmed.inertia_)
    return np.vstack(centroids)
from scipy.spatial.distance import cdist
def similarity(fps,centroids,metric):
  if len(fps)>0:
    sims=cdist(fps,centroids,metric)
    return 1-np.mean(np.min(sims,axis=1)),np.min(sims,axis=1)
  else:
    return 0,0

for el in ["<RX_10>","<RX_1>","<RX_2>","<RX_3>","<RX_4>","<RX_5>","<RX_6>","<RX_7>","<RX_8>","<RX_9>"]:
  df_rxn_10=df[(df["reaction_type"]==el)].copy()
  train_inputs,train_input_len,df_rxn_10_max_200=tokenize_and_pad(df_rxn_10,200)
  fingerprints=[]
  for i in df_rxn_10_max_200.index:
    array=np.zeros((0, ), dtype=np.float64)
    Chem.DataStructs.ConvertToNumpyArray(df_rxn_10_max_200.loc[i,"fingerprint"], array)
    fingerprints.append(array)

  centroids=find_centroids(np.array(fingerprints))
  centroids_strings=[]
  for i in centroids:
    select=[np.array_equal(i,j) for j in fingerprints]
    if(np.sum(select))>0:
      centroids_strings.append(train_inputs[select][0])
  print(el,similarity(fingerprints,centroids,"jaccard")[0])
  print(el,similarity(train_inputs,centroids_strings,"cosine")[0])
"""

removed  0  samples
181 9


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


inertia: 84.64804907413179
<RX_10> 0.4502129246939205
<RX_10> 0.7959826731307652
removed  93  samples
12005 600


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


inertia: 7209.851408916508
<RX_1> 0.2647825602319377
<RX_1> 0.8545752868265527
removed  183  samples
9348 467


...

inertia: 1946.0618606799903
<RX_7> 0.36565196419899726
<RX_7> 0.8547592929451419
removed  5  samples
646 32


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


inertia: 372.4908224078514
<RX_8> 0.3287177408055372
<RX_8> 0.836979295515362
removed  10  samples
1457 72


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


inertia: 911.5870378703984
<RX_9> 0.285955077156394
<RX_9> 0.8390922577237193


Now let's create the pipeline for validation and test data:

In [ ]:
def data_to_Ids_pipeline(reactants_df, products_df, max_len=0):
    dfa = process_dfs(reactants_df, products_df)
    prepare_whole_reaction_padded(dfa)
    inputs, input_len, dfa_new = tokenize_and_pad(dfa, max_len)
    return dfa_new, inputs, input_len

In [ ]:
valid_products_df = pd.read_csv(os.path.join(
    validation_path, "valid_sources.txt"), header=None)
valid_reactants_df = pd.read_csv(os.path.join(
    validation_path, "valid_targets.txt"), header=None)

test_products_df = pd.read_csv(os.path.join(
    test_path, "test_sources.txt"), header=None)
test_reactants_df = pd.read_csv(os.path.join(
    test_path, "test_targets.txt"), header=None)
valid_df, valid_inputs, valid_input_len = data_to_Ids_pipeline(
    valid_reactants_df, valid_products_df, max_len=200)

test_df, test_inputs, test_input_len = data_to_Ids_pipeline(
    test_reactants_df, test_products_df, max_len=200)
print("Shape of validation Ids inputs:", valid_inputs.shape)
print("Shape of Test Ids inputs:",  test_inputs.shape)

In [ ]:
with open(os.path.join(train_path, "train_targets_ids_200.data"), "w") as f:
    np.savetxt(f, train_inputs, newline='\n')
with open(os.path.join(validation_path, "validation_targets_ids_200.data"), "w") as f:
    np.savetxt(f, valid_inputs, newline='\n')
with open(os.path.join(test_path, "test_targets_ids_200.data"), "w") as f:
    np.savetxt(f, test_inputs, newline='\n')

with open(os.path.join(train_path, "train_targets_ids_200.txt"), "w") as f:
    np.savetxt(f, train_inputs, newline='\n')
with open(os.path.join(validation_path, "validation_targets_ids_200.txt"), "w") as f:
    np.savetxt(f, valid_inputs, newline='\n')
with open(os.path.join(test_path, "test_targets_ids_200.txt"), "w") as f:
    np.savetxt(f, test_inputs, newline='\n')

with open(os.path.join(train_path, "train_targets_rxns_200.data"), "w") as f:
    np.savetxt(f, df_with_max_200["reaction_type"].str.replace("<RX_", "").str.replace(
        ">", "").apply(lambda x: 0 if int(x) == 10 else int(x)).values, newline='\n')
with open(os.path.join(train_path, "train_targets_ids_fingerprints_200.data"), "w") as f:
    np.savetxt(f, fingerprints, newline='\n')

#### Using Kmedoids to select the clusters representative of each class:

In [ ]:
import numpy as np
from sklearn_extra.cluster import KMedoids
from rdkit import Chem


def find_centroids(clusters, fps):
    unique_clusters = np.unique(clusters)
    centroids = []
    for cluster in unique_clusters:
        cluster_els = fps[np.where(clusters == cluster)]
        nb_of_clusters = int(np.floor(len(cluster_els)*0.05))
        print(len(cluster_els), nb_of_clusters)
        kmed = KMedoids(metric='jaccard', n_clusters=nb_of_clusters,
                        random_state=48, init='k-medoids++').fit(cluster_els)
        centroids.append(kmed.cluster_centers_)
        print("inertia:", kmed.inertia_)
    return np.vstack(centroids)


reactions_nb = [int(t.replace("<RX_", "").replace(">", ""))
                for t in df_with_max_200["reaction_type"]]
centroids = find_centroids(reactions_nb, np.array(fingerprints))
centroids_strings = []
for i in centroids:
    select = [np.array_equal(i, j) for j in fingerprints]
    if (np.sum(select)) > 0:
        centroids_strings.append(train_inputs[select][0])
        print(len(centroids_strings))
print(len(centroids_strings))

12009 600


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1738: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1738: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


inertia: 7214.776284755243
9351 467


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1738: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1738: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


inertia: 5427.790960606616
4448 222


...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1738: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1738: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1738: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


inertia: 372.4908224078514
1457 72


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1738: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


inertia: 911.5870378703984
181 9
inertia: 84.64804907413179


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1738: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1738: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [ ]:
with open(os.path.join(train_path, "centroids_200.data"), "w") as f:
    np.savetxt(f, centroids, newline='\n')
with open(os.path.join(train_path, "centroids_strings_200.data"), "w") as f:
    np.savetxt(f, centroids_strings, newline='\n')